In [5]:
from pymilvus import MilvusClient
from llm import StarEncoder, CodeBERT
import torch
from llm2vec import LLM2Vec
from huggingface_hub import login
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from evaluate import evaluate

# Database connection and setup

In [7]:
client = MilvusClient(uri="http://localhost:19530", user="root", password="Milvus")

In [18]:
models = ["bert", "starcoder"]
for current_model in models:
    collection_name = current_model

    client.create_collection(
        collection_name=collection_name,
        dimension=768,
        auto_id=True,
    )
client.create_collection(
    collection_name="llama",
    dimension=2048,
    auto_id=True,
)

KeyboardInterrupt: 

# Refresh databases

In [ ]:
models = ["bert", "starcoder"]
for current_model in models:
    collection_name = current_model
    client.drop_collection(
        collection_name=collection_name,
    )
    client.create_collection(
        collection_name=collection_name,
        dimension=768,
        auto_id=True,
    )

client.drop_collection(
    collection_name="llama",
)
client.create_collection(
    collection_name="llama",
    dimension=2048,
    auto_id=True,
)

# Evaluate

In [23]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

EVAL_CONFIGS = {
    "starencoder": {
        "max_input_len": 10000,
        "maximum_token_len": 1024,
        "device": DEVICE,
    },
    "codebert": {
        "max_input_len": 10000,
        "maximum_token_len": 512,
        "device": DEVICE,
    },
}

In [3]:
dataset = load_dataset("D:\\CSN")

# Bert

In [24]:
bert = CodeBERT(**EVAL_CONFIGS.get("codebert"))

In [25]:
evaluate(bert, client, dataset, "bert")
print("done")

  0%|          | 0/26909 [00:00<?, ?it/s]


AttributeError: 'list' object has no attribute 'detach'

# StarEncoder

In [ ]:
starencoder = StarEncoder(**EVAL_CONFIGS.get("starencoder"))

In [ ]:
evaluate(starencoder, client, dataset, "starcoder")
print("done")

  4%|▎         | 1000/26909 [00:29<12:51, 33.58it/s]


insert complete


  4%|▎         | 1000/26909 [00:35<15:30, 27.85it/s]

Search complete
0.36582743815493596
done


# Llama-2

In [5]:
from llm2vec import LLM2Vec

import torch
from transformers import AutoTokenizer, AutoModel, AutoConfig
from peft import PeftModel

# Loading base Mistral model, along with custom code that enables bidirectional connections in decoder-only LLMs. MNTP LoRA weights are merged into the base model.
tokenizer = AutoTokenizer.from_pretrained("McGill-NLP/LLM2Vec-Sheared-LLaMA-mntp")
config = AutoConfig.from_pretrained(
    "McGill-NLP/LLM2Vec-Sheared-LLaMA-mntp", trust_remote_code=True
)
llama = AutoModel.from_pretrained(
    "McGill-NLP/LLM2Vec-Sheared-LLaMA-mntp",
    trust_remote_code=True,
    config=config,
    torch_dtype=torch.bfloat16,
    device_map="cuda" if torch.cuda.is_available() else "cpu",
)
llama = PeftModel.from_pretrained(
    llama,
    "McGill-NLP/LLM2Vec-Sheared-LLaMA-mntp",
)
llama = llama.merge_and_unload()  # This can take several minutes on cpu

# Loading supervised model. This loads the trained LoRA weights on top of MNTP model. Hence the final weights are -- Base model + MNTP (LoRA) + supervised (LoRA).
llama = PeftModel.from_pretrained(
    llama, "McGill-NLP/LLM2Vec-Sheared-LLaMA-mntp-supervised"
)

# Wrapper for encoding and pooling operations
l2v = LLM2Vec(llama, tokenizer, pooling_mode="mean", max_length=512)

Some weights of the model checkpoint at princeton-nlp/Sheared-LLaMA-1.3B were not used when initializing LlamaEncoderModel: ['lm_head.weight']
- This IS expected if you are initializing LlamaEncoderModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LlamaEncoderModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:
class LLM2VecWrapper:
    def __init__(self, model):
        self.model = model

    def encode(self, input):
        encoding = self.model.encode(input, show_progress_bar=False)
        return torch.nn.functional.normalize(encoding, p=2, dim=1).detach().numpy()

In [28]:
evaluate(LLM2VecWrapper(l2v), client, dataset, "llama")
print("done")

  4%|▎         | 1000/26909 [03:38<1:34:10,  4.59it/s]

Search complete
0.8241809795430913
done


In [ ]:
from transformers.utils.logging import disable_progress_bar
import datasets
import os

from tqdm import tqdm
from functools import partialmethod

tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

os.environ["TQDM_DISABLE"] = "1"
disable_progress_bar()
datasets.disable_progress_bar()
l2v.encode("test")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

tensor([[-1.7109, -0.3945,  0.7812,  ...,  0.9297,  1.8594,  0.2021],
        [-1.7109,  0.4961,  1.6328,  ...,  1.1094, -0.2148, -0.5859],
        [-1.8438, -0.2061,  1.0234,  ...,  1.1094, -0.5156,  0.2539],
        [-1.7109, -0.3945,  0.7812,  ...,  0.9297,  1.8594,  0.2021]])

In [ ]:
import transformers

transformers.logging.set_verbosity_error()

ranks = []
for idx, example in enumerate(tqdm(dataset["test"].shuffle(seed=42))):

    # print(example["func_code_string"])
    query_vector = l2v.encode([example["func_documentation_string"]])

    q_reps_norm = torch.nn.functional.normalize(query_vector, p=2, dim=1)

    res = client.search(
        collection_name="llama",  # target collection
        data=[q_reps_norm[0].numpy()],  # query vectors
        limit=1000,  # number of returned entities
        output_fields=["text"],  # specifies fields to be returned
        search_params={"metric_type": "COSINE"},
    )

    for index, result in enumerate(res[0]):
        if example["func_code_string"] == result["entity"]["text"]:
            ranks.append(1 / (index + 1))

    if idx == 1000:
        break
print("MRR = ", np.mean(ranks))
print("done")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

MRR =  0.8296693368434425
done


In [ ]:
evaluate(t5, client, dataset, "t5")

0.8296693368434425
